In [ ]:
# 필수 라이브러리 임포트
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaForSequenceClassification, 
    RobertaTokenizer, 
    AutoModelForCausalLM, 
    AutoTokenizer, 
    GenerationConfig, 
    pipeline, 
    TextStreamer, 
    TextIteratorStreamer
)
import pandas as pd
import numpy as np
import json
import faiss
from boltons.iterutils import chunked
from tqdm import tqdm
from keybert import KeyBERT
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import langchain
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryWithErrorOutputParser
from langchain.output_parsers.format_instructions import PYDANTIC_FORMAT_INSTRUCTIONS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from auto_gptq import AutoGPTQForCausalLM
from functools import partial
from threading import Thread
from typing import List, Mapping, Optional, Any, Dict
from pydantic import BaseModel, Field, validator

In [ ]:
# CUDA 사용 가능 여부 확인
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
pd.set_option('display.max_colwidth', None)

# (1) Text Emotion Recognition

In [ ]:
# 감정 라벨 정의
emotion_to_label = {
    0: 'anger', 1: 'disgust', 2: 'fear', 3: 'sadness', 4: 'contentment', 
    5: 'excitement', 6: 'awe', 7: 'amusement'
}

# RoBERTa 모델 및 토크나이저 불러오기
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(emotion_to_label))
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# 모델을 GPU로 이동
model.to(DEVICE)
model.eval()

# 모델 파일 경로
model_path = "./0ver2_SOTA_ED_model_comp_0.8973.pt"

# 저장된 모델 불러오기
state_dict = torch.load(model_path)

# 모델에 state_dict 적용
model.load_state_dict(state_dict)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = pd.DataFrame(data)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        utterance = self.data['utterance'].iloc[idx]
        text = f"{utterance}"

        encoding = self.tokenizer(text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask']
        }

In [ ]:
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv("./example_data.csv")

df_dataset = EmotionDataset(df, tokenizer)

In [ ]:
# 예측 수행
def predict_label(inputs):
    with torch.no_grad():
        inputs['input_ids'] = inputs['input_ids'].to(DEVICE)
        inputs['attention_mask'] = inputs['attention_mask'].to(DEVICE)
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = logits.argmax(dim=1).item()
    predicted_label = emotion_to_label[predicted_label_id]
    return predicted_label

In [ ]:
predictions = [predict_label(inputs) for inputs in df_dataset]

df['emotion'] = predictions
print(df['utterance'].iloc[0])
print(df['emotion'].iloc[0])

My wife made me pancakes for breakfast. I have a full belly and feel rather happy now.
contentment


# <font color='red' size='60pt'> 사용할 예시 텍스트: </font>

## My wife made me pancakes for breakfast. I have a full belly and feel rather happy now.

# <font color='red' size='60pt'> 예측된 감정: </font>

## <font color='red'> Contentment </font>

# (2) Tokenization and Keyphrase Extraction

In [ ]:
# keybert로 key phrase 추출
# KeyBERT 모델 초기화
kw_model = KeyBERT()

def filter_pos_with_stopwords(text):
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1,5),use_maxsum = True,top_n = 5)
  sorted_keywords = sorted(keywords, key=lambda x: x[1], reverse=True)
  return sorted_keywords[0][0]

df['keyphrase'] = df['utterance'].apply(filter_pos_with_stopwords)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


In [ ]:
# 불용어 제거 전 축약어 전처리
def expand_contractions(text):
    contractions = {
        "he's": "he is",
        "He's": "He is",
        "she's": "she is",
        "She's": "She is",
        "I've": "I have",
        "i've": "i have",
        "you've": "you have",
        "You've": "You have",
        "we've": "we have",
        "We've": "We have",
        "they've": "they have",
        "They've": "They have",
        "she'll": "she will",
        "She'll": "She will",
        "he'll": "he will",
        "He'll": "He will",
        "don't": "do not",
        "Don't": "Do not",
        "doesn't": "does not",
        "Doesn't": "Does not",
        "didn't": "did not",
        "Didn't": "Did not",
        "shouldn't": "should not",
        "Shouldn't": "Should not",
        "wouldn't": "would not",
        "Wouldn't": "Would not",
        "I'll": "I will",
        "i'll": "i will",
        "i'd": "i would",
        "I'd": "I would",
        "you'd": "you would",
        "You'd": "You would",
        "we'd": "we would",
        "We'd": "We would",
        "they'd": "they would",
        "They'd": "They would",
        "i'm": "i am",
        "I'm": "I am",
        "you're": "you are",
        "You're": "You are",
        "it's": "it is",
        "It's": "It is",
        "we're": "we are",
        "We're": "We are",
        "they're": "they are",
        "They're": "They are",
        "isn't": "is not",
        "Isn't": "Is not",
        "aren't": "are not",
        "Aren't": "Are not",
        "wasn't": "was not",
        "Wasn't": "Was not",
        "weren't": "were not",
        "Weren't": "Were not",
        "hasn't": "has not",
        "Hasn't": "Has not",
        "haven't": "have not",
        "Haven't": "Have not",
        "hadn't": "had not",
        "Hadn't": "Had not",
        "won't": "will not",
        "Won't": "Will not",
        "wouldn't": "would not",
        "Wouldn't": "Would not",
        "can't": "can not",
        "Can't": "Can not",
        "couldn't": "could not",
        "Couldn't": "Could not",
        "shouldn't": "should not",
        "Shouldn't": "Should not",
        "mightn't": "might not",
        "Mightn't": "Might not",
        "mustn't": "must not",
        "Mustn't": "Must not",
        "there's": "there is",
        "There's": "There is",
        "they're": "they are",
        "They're": "They are",
        "who's": "who is",
        "Who's": "Who is",
        "what's": "what is",
        "What's": "What is",
        "where's": "where is",
        "Where's": "Where is",
        "here's": "here is",
        "Here's": "Here is",
        "when's": "when is",
        "When's": "When is",
        "why's": "why is",
        "Why's": "Why is",
        "how's": "how is",
        "How's": "How is",
        "ain't": "are not",
        "can't've": "cannot have",
        "cause": "because",
        "could've": "could have",
        "couldn't've": "could not have",
        "hadn't've": "had not have",
        "hasn't've": "has not have",
        "haven't've": "have not have",
        "he'd've": "he would have",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "I would",
        "i'd've": "I would have",
        "i'll": "I will",
        "i'll've": "I will have",
        "i'm": "I am",
        "i've": "I have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "wen't": "were not",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have",
        "!": ",",
        "....": ",",
        "...": ",",
        "..": ",",
        ".": ",",
        "*": " ",
        "-": " ",
        ",,,,,": ",",
        ",,,,": ",",
        ",,,": ",",
        ",,": ",",

    }

    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

df['T5_T2'] = df['utterance'].apply(expand_contractions)

In [ ]:
# 불용어 제거
# NLTK 불용어 목록을 불러오기
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# 불용어 제거 함수
def remove_stopwords(text):
    # 텍스트 소문자로 변환
    text = text.lower()
    word_tokens = word_tokenize(text)
    filtered_sentence = [word for word in word_tokens if not word in stop_words]
    return ' '.join(filtered_sentence)

# 'utterance' 열에 불용어 제거 함수 적용
df['T5_T2'] = df['T5_T2'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/a2070064/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/a2070064/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# 특수문자 제거
def expand_contractions2(text):
    contractions = {
        ",,,,,,": ",",
        ",,,,,": ",",
        ",,,,": ",",
        ",,,": ",",
        ",,": ",",
        ",  ,": ",",
        ", ,": ",",
        "    ": " ",
        "   ": " ",
        "  ": " ",
        " ,": ",",
    }
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

df['T5_T2'] = df['T5_T2'].apply(expand_contractions2)

In [ ]:
df['E1_T5_T2'] = df['emotion'] + ", " + df['T5_T2'] + ", " + df['keyphrase']
print(df['T5_T2'].iloc[0] + ", " + df['keyphrase'].iloc[0])
print(df['E1_T5_T2'].iloc[0])

wife made pancakes, breakfast, full belly, feel rather happy, made pancakes
contentment, wife made pancakes, breakfast, full belly, feel rather happy, made pancakes


# <font color='red' size='60pt'> 토큰화 및 중요 어구 추출 후 Append: </font>

## wife made pancakes, breakfast, full belly, feel rather happy, made pancakes

# <font color='red' size='60pt'> 감정 라벨 Prepend: </font>

## <font color='red'>contentment</font>, wife made pancakes, breakfast, full belly, feel rather happy, made pancakes


# (3) Append 3 synonyms

In [ ]:
# JSON 파일에서 동의어 데이터 불러오기
with open('./emotion_synonyms.json', 'r', encoding='utf-8') as file:
    synonyms = json.load(file)

In [ ]:
from angle_emb import AnglE

# 사전 학습된 AnglE 모델을 불러오기
uae = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

In [ ]:
# AnglE 이용해 벡터로 인코딩, faiss 인덱스를 설정
def get_index(emotion):
    np.random.seed(1234)
    d = 1024  # 벡터의 차원 수
    index = faiss.IndexFlatL2(d) # L2 거리(유클리드 거리) 메트릭을 사용하는 faiss 인덱스 초기화

    chunk_size = 64
    synonyms_list = synonyms.get(emotion, [])  # 해당 감정에 대한 동의어 목록 불러오기

    # 동의어 목록을 청크 단위로 나누어 인코딩 후 Faiss 인덱스에 벡터를 추가
    for chunk in tqdm(chunked(synonyms_list, chunk_size)):
        for vec in uae.encode(chunk):
            index.add(vec.reshape(-1, d))

    return index

In [ ]:
# 감정에 대한 faiss 인덱스에서 입력 벡터와 가장 유사한 상위 3개의 벡터를 검색하는 함수
def get_top3_synonyms(emotion, text):
    vec = uae.encode(text)  # 입력 텍스트를 벡터로 인코딩
    synonyms_list = synonyms.get(emotion, [])
    append_synonyms = ''  # 결과 동의어 문자열 초기화

     # 입력 벡터와 가장 유사한 상위 3개의 벡터 검색
    D, I = get_index(emotion).search(vec, 3) # D는 검색된 거리를, I는 해당 인덱스의 ID를 반환
    for id in I[0]:
      if append_synonyms=='':
        append_synonyms += synonyms_list[id]
      else: append_synonyms += ', ' + synonyms_list[id]

    print(append_synonyms)

    return append_synonyms

In [ ]:
df['E1_T5_T2_E4'] = df.apply(lambda row: f"{row['E1_T5_T2']}, {get_top3_synonyms(row['emotion'], row['utterance'])}", axis=1)
print(df['E1_T5_T2_E4'].iloc[0])

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 56.09it/s]

gratification, satiation, light heartedness
contentment, wife made pancakes, breakfast, full belly, feel rather happy, made pancakes, gratification, satiation, light heartedness


# <font color='red' size='60pt'> 감정 라벨의 동의어 중 원본 문장과 유사도 높은 3개 Append: </font>

## contentment, wife made pancakes, breakfast, full belly, feel rather happy, made pancakes <font color='red'> gratification, satiation, light heartedness</font>

# (4) Append Style Modifiers

In [ ]:
# 임베딩 모델 다운로드 및 설정
def load_embeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"):
    return HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
# Chroma 데이터베이스 초기화
def initialize_chroma(persist_directory, embeddings):
    return Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
# Llama 모델 및 토크나이저 로드
def load_model_and_tokenizer(model_name_or_path, model_basename, device):
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        revision="gptq-4bit-128g-actorder_True",
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        inject_fused_attention=False,
        device=device,
        quantize_config=None,
    )
    return model, tokenizer


In [ ]:
# 프롬프트 생성 함수
def generate_prompt(prompt, system_prompt=""):
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
# 텍스트 생성 파이프라인 설정
def initialize_text_pipeline(model, tokenizer, streamer):
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        temperature=0,
        top_p=0.95,
        repetition_penalty=1.15,
        streamer=streamer,
    )

In [ ]:
# 주어진 LLama 파이프라인, 데이터베이스, 시스템 프롬프트 템플릿으로 QA 체인을 초기화
def initialize_qa_chain(llm, db, system_prompt_template):
    prompt = PromptTemplate(template=system_prompt_template, input_variables=["context", "question"])
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 5}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )

In [ ]:
# 시스템 프롬프트 템플릿
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

# 사용자 정의 프롬프트 템플릿
USER_PROMPT_TEMPLATE = generate_prompt(
    """
{context}

Instruction: Give style modifiers for generating the image of the input TEXT.
1. From the given context, select style modifiers to improve the emotion expression, aesthetics and authenticity of the final image.
2. Output style modifiers in 3 or less.

TEXT: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
# 초기화
embeddings = load_embeddings()
db = initialize_chroma(persist_directory="./styleDB", embeddings=embeddings)

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model, tokenizer = load_model_and_tokenizer(model_name_or_path, model_basename, device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
text_pipeline = initialize_text_pipeline(model, tokenizer, streamer)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})
qa_chain = initialize_qa_chain(llm, db, USER_PROMPT_TEMPLATE)

# # 데이터프레임에 새 열 추가
df['answer'] = ""
df['modifiers'] = ""

for index, row in df.iterrows():
    answer_format = '\n* Use this format(JSON) for the answer {\"modifiers\": []}.' # 답변 포맷 정의
    text = row['E1_T5_T2_E4']
    df.at[index, 'answer'] = qa_chain(f"{text}{answer_format}").get('result') # QA 체인을 통해 답변 생성 및 데이터프레임에 저장

print(df['answer'].iloc[0])

/home/a2070064/demo/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/a2070064/demo/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Based on the given text, here are three style modifiers that could be used to generate an image that captures the emotions and themes present in the text:

1. Warm colors: To convey a sense of comfort, happiness, and satisfaction, warm colors such as golden yellow, orange, and red can be used. These colors can be applied to the background, objects, or even the characters' skin tones to create a cozy and inviting atmosphere.
2. Soft brush strokes: To emphasize the lightheartedness and playfulness of the scene, soft brush strokes can be used to give the impression of gentle, effortless movements. This can also help to convey a sense of ease and contentment.
3. Whimsical patterns: To add a touch of fun and playfulness to the image, whimsical patterns such as polka dots, stripes, or other geometric shapes can be incorporated into the design. These patterns can be used for the characters' clothing, the background, or even the utensils and plates used in the breakfast scene.

Here is the ou

In [ ]:
df['answer'].iloc[0]

'[INST] <>\nUse the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n<>\n\n\n{"description": "The JSON file contains style-modifiers list categorized as \'color\', \'dimensionality\', \'light\', \'perspective\', and \'style\'. In each category, modifiers are separated by commas.", "modifiers": [{"color": ["White", "Black", "Brown", "Light Gray", "Gray", "Dark Gray", "Maroon", "Red", "Orange", "Yellow", "Lime", "Green", "Cyan", "Teal", "Blue", "Indigo", "Purple", "Violet", "Fuchsia", "Magenta", "Pink", "Tan", "Beige", "Blush", "Scarlet", "Olive Green", "Chartreuse", "Turquoise", "Aqua", "Azure", "Dark White", "Dark Brown", "Dark Maroon", "Dark Red", "Dark Orange", "Dark Yellow", "Dark Lime", "Dark Green", "Dark Cyan", "Dark Blue", "Dark Purple", "Dark Magenta", "Dark Pink", "Light Black", "Light Brown", "Light Maroon", "Light Red", "Light Orange", "Light Yellow", "Light Lime", "

In [ ]:
def get_array (text) :
    index = text.find("[/INST]")
    if index != -1:  # 찾은 경우
        text = text[index + len("[/INST]"):]
        
    start_index = text.find('[')

    end_index = text.rfind(']') + 1

    # 배열 부분 추출
    array_part = text[start_index:end_index].strip()

    return array_part

In [ ]:
df['modifiers'] = df['answer'].apply(get_array)
print(df['modifiers'].iloc[0])

[{"type": "color", "value": ["golden yellow", "orange", "red"]}, {"type": "brush stroke", "value": ["soft", "effortless"]}, {"type": "pattern", "value": ["polka dots", "stripes", "geometric shapes"]}]


In [ ]:
def get_modifiers (string) :
    # 제거할 요소들
    elements_to_remove = ["modifiers", "artist", "color", "dimensionality", "style", "light", "modifier", "category", "perspective", "value", "typography", "type", "hand-written", "textual"]
    string = string.replace("\\n", '')
    try:
        json_data = json.loads(string)
        values = [list(obj.values())[1] for obj in json_data]
        values = [item for item in values if item not in elements_to_remove]
        result_string = ", ".join(values)
        result_string = re.sub(r'(?i)typography', '', result_string)
        return result_string
    except Exception as e:
        print(f'org: {string}')

        # 제거할 요소들을 문자열에서 삭제하는 반복문
        for element in elements_to_remove:
            string = string.replace('"' + element + '"', ' ')

        # 제거할 문자들
        characters_to_remove = r'[,:\[\]\{\}\\;]'

        # 문자열에서 지정된 문자들을 모두 제거
        string = re.sub(characters_to_remove, '', string)

        extracted_list = re.findall(r'"(.*?)"', string)
        # 추출된 값을 필터링하여 요소 제거
        values = [item.strip() for item in extracted_list if item.lower().strip() not in elements_to_remove]

        result_string = ", ".join(values)
        result_string = re.sub(r'(?i)typography', '', result_string)

        print(result_string)

        return result_string

In [ ]:
df['modifiers'] = df['modifiers'].apply(get_modifiers)

org: [{"type": "color", "value": ["golden yellow", "orange", "red"]}, {"type": "brush stroke", "value": ["soft", "effortless"]}, {"type": "pattern", "value": ["polka dots", "stripes", "geometric shapes"]}]
golden yellow, orange, red, brush stroke, soft, effortless, pattern, polka dots, stripes, geometric shapes


In [ ]:
style_list = list(df['modifiers'].iloc[0].split(','))
style_list = str(style_list[0] +","+ style_list[3] +","+ style_list[4])
style_list

'golden yellow, brush stroke, soft'

In [ ]:
df['prompt'] = df['E1_T5_T2_E4'] + ', ' + style_list
print(df['prompt'].iloc[0])

contentment, wife made pancakes, breakfast, full belly, feel rather happy, made pancakes, gratification, satiation, light heartedness, golden yellow, brush stroke, soft


# <font color='red' size='60pt'> 감정 라벨과 연관 높은 스타일 키워드 3개 Append: </font>

## contentment,wife made pancakes, breakfast, full belly, feel rather happy, made pancakes, gratification, satiation, light heartedness, <font color='red'>golden yellow, brush stroke, soft
</font>